In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error

from utilidades.validar import validar

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

dfinicial = pd.read_csv("data/train_limpio.csv")
dftest = pd.read_csv("data/test_limpio.csv")

#dfinicial2 = pd.read_csv("data/esteban/train_limpio_ciudades.csv")
#dftest2 = pd.read_csv("data/esteban/test_limpio_ciudades.csv")

xg = pd.read_csv('submits/submit_xg_prueba.csv')

In [3]:
xg.head()

,id,target
0,4941,4831603.00
1,51775,812344.94
2,115253,2203735.00
3,299321,1403080.20
4,173570,596104.40


In [4]:
pd.set_option('display.max_columns', 23) #ver todas las columnas
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

In [5]:
def generar_subida(DFINICIAL, metodo, DFTESTEAR, nomArchivo, pasarAdisco):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
    X, y = DFINICIAL.drop( columns = ['precio'] ), DFINICIAL["precio"]

    reg = metodo(
        alpha=0.9, #no cambia nada
        max_depth=4,
        n_estimators=5,
        learning_rate=0.5,
        min_samples_leaf=250,
        subsample=0.5,
        validation_fraction=0.1
    )
    reg.fit(X,y)
    
    subida = pd.DataFrame( reg.predict(DFTESTEAR) , columns=["target"])
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    # Heurística: si vale menos de 310 000, ponerle 310 000
    subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    
    # Heurística 2: si vale más de 12 500 000, ponerle 12 500 000
    subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    #Este modelo no necesita las euristicas,pero igual no da muy bien
    
    
    if pasarAdisco:
        subida.to_csv(f"submits/" + nomArchivo, index = False) 
    
    return X, y, subida

In [31]:
dfinicial1 = dfinicial[dfinicial['metrostotales']<250]
dftest1 = dftest.drop( columns = ['metrostotales'] ) #no uso este,uso dftest,CON metrostotales

#dfinicial1 = dfinicial2.drop( columns = [ 'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos',  'escuelascercanas'
#                           ])                                        
 
#dftest1 = dftest2.drop( columns = [  'anio', 'metrostotales', 'dia', 'mes', 'banos', 'centroscomercialescercanos', 'escuelascercanas'
#                           ])   

X, y, subida = generar_subida( dfinicial1, GradientBoostingRegressor, dftest, "prueba_andres.csv", True)

# IMPRIME EL PROMEDIO DE DIFERENCIA CON EL MEJOR RESULTADO (EL DE XG DE JULIÁN)
# UN VALOR MENOR ES MEJOR
dftot = subida.merge(xg, on = 'id', how = 'inner')
dftot['diferencia'] = abs( dftot['target_x'] - dftot['target_y'] )
round (dftot['diferencia'].sum() / len( dftot['diferencia'] ) )

361642.0

In [24]:
#Sólo si el valor anterior es menor a 500 000, usar esto:
validar(X, y, GradientBoostingRegressor, False)

Error de fold N°0 es : 712995.6947552349
Error de fold N°1 es : 705309.1755835403
Error de fold N°2 es : 705762.1727442549
Error de fold N°3 es : 711824.3224002372
Error de fold N°4 es : 717253.4112520895


In [ ]:
#reg = LinearRegression(normalize=True, n_jobs = -1)
#reg.fit(X,y)

In [ ]:
dfinicial1.head()